---
title: Numerical Integration in Julia
venue: Modules
---

<b> QuadGK </b> package for 1D integration: https://juliamath.github.io/QuadGK.jl/stable/

In [ ]:
using QuadGK, Plots,Plotly, ApproxFun, SymbolicNumericIntegration

Automatic numeric evaluation of a simple integral with adaptive <b> Gauss-Konrod </b> scheme, only need to specify function and interval boundaries: $\int_{-1}^1 5x^4 \ dx= \left[x^5\right]_{-1}^1=2$

In [ ]:
f(x)=5x^4
#Define endpoints of integration interval [a, b]
interval_a=-1
interval_b=1
result, error = quadgk(f, interval_a, interval_b)

Get quadrature points and weights for <b> Gauss quadrature </b>:

In [ ]:
#Specify number of weights
N=3
x,weights=gauss(N, interval_a, interval_b)

Calculate integral with quadrature points and weights: $\int_a^b f(x) \ dx \approx \sum w_i\cdot  f(x_i)$ 

Visualize the Gauss quadrature points:

In [ ]:
xfine=(-1:0.01:1)
trace1=Plotly.scatter(x=x,y=f.(x),mode="markers",name="Quadrature points")
trace2=Plotly.scatter(x=xfine,y=f.(xfine),mode="line",name="f(x)")
Plotly.plot([trace1,trace2])

In [ ]:
result=sum(weights.*f.(x))

<b> Number of Gauss quadrature points vs. accuracy for polynomials of different orders </b>

In [ ]:
function gquad_error(order,NQuad)
        #f(x) =x^order, order should be even
        f(x)=(order+1)*x^order
        x,weights=gauss(NQuad, interval_a, interval_b)
        result=quadgk(f,interval_a,interval_b)[1]-sum(weights.*f.(x))
end

Let $n=2m$ be the even order of a polynomial function $f(x)=x^n$. Then a minimum of $n/2+1=m+1$ Gauss quadrature points are needed to compute the integral exactly on the interval $[-1,1]$ (to within machine precision). If $n=2m+1$ is odd, then $1$ single quadrature point is needed.

In [ ]:
N_order=16
N_QPoints=8
gquad_error(N_order,N_QPoints)

<b> Comparison with other integration schemes </b>

A general numerical integration scheme, also called quadrature, is based on the evaluation of the function $f(x)$ at a finite number of points $x_0=a,\dots,x_n=b$ within the integration interval $[a,b]$. It  can be written as:

\begin{equation}
    \int_a^b f(x)\, dx \approx h \sum_{i=0}^n w_if(x_i) =  h \bigl[ w_0f(x_0)+w_1f(x_1)+\cdots w_nf(x_n) \bigr],
  \end{equation}

<b> Example: Trapezoid formula </b> (https://tobydriscoll.net/fnc-julia/localapprox/integration.html) 

\begin{split}
  \int_a^b f(x)\, dx \approx T_f(n) &= h\left[
    \frac{1}{2}f(x_0) + f(x_1) + f(x_2) + \cdots + f(x_{n-1}) +
    \frac{1}{2}f(x_n) \right].
\end{split}

In [ ]:
 """
     trapezoid(f,a,b,n)
 
 Apply the trapezoid integration formula for integrand `f` over
 interval [`a`,`b`], broken up into `n` equal pieces. Returns
 the equidistant quadrature points, and the quadrature weights on these points (0.5 for the first and
 last point, one for all interior points)
"""
 function trapezoid(f,a,b,n)
    h = (b-a)/(n+1)
    x = range(a,b,length=n+2)
      weights =ones(1,n+2)*h
      weights[1]*=0.5
      weights[n+2]*=0.5
   
    return x,weights
end

Compare Trapezoid scheme with Gauss quadrature  for $\int_{-1}^1 7x^6 \ dx =2 $ using the same number of interior points:

In [ ]:
f6(x)=7*x^6

In [ ]:
#Fix number of interior points for both Gauss and Trapezoid
N_interior=16

#Get quadrature points and weights for the trapezoid scheme
x_t,weights_t =trapezoid(f6,interval_a,interval_b,N_interior)
#Gauss
x_g,weights_g=gauss(N_interior, interval_a, interval_b)

xfine=(-1:0.01:1)
trace0=Plotly.scatter(x=x_g,y=f6.(x_g),mode="markers",name="Gauss quadrature points")
trace1=Plotly.scatter(x=x_t,y=f6.(x_t),mode="markers",name="Trapezoid quadrature points")
trace2=Plotly.scatter(x=xfine,y=f6.(xfine),mode="line",name="f(x)")
Plotly.plot([trace0,trace1,trace2])

In [ ]:
#Trapezoid error
result_trapezoid=sum(weights_t*f6.(x_t))
error_trapezoid=abs(result_trapezoid-2.0)

#Gauss error

result_gauss=sum(weights_g.*f6.(x_g))
error_gauss=abs(result_gauss-2.0)

print("Using ", N_interior, " quadrature points, error for Trapezoid is ",error_trapezoid,", error for Gauss is ", error_gauss)

<b> Multi-dimensional integration: Cubature package</b> (https://github.com/JuliaMath/Cubature.jl) 

<b> Example: </b> $\int_{[0,1]\times[0,1]} 4xy\  dx dy = \int_0^1 \int_0^1 4xy\ dx dy =\int_0^1 \left[2yx^2 \right]_{x=0}^{x=1} dy = \int_0^1 2y dy  = 1 $

In [ ]:
using Cubature

In [ ]:
#Define function on 2D box
f2d(x)=4*x[1]*x[2]

#Define integration boundaries
xmin=[0 0]
xmax=[1 1]

In [ ]:
result,error=hcubature(f2d, xmin, xmax)

# In-Class Exercise

<b> Simpson's integration formula</b> with $n-1=2m+1, m\geq 1 $ interior points is given by

  \begin{split}
    \int_a^b f(x)\, dx \approx  \frac{h}{3}\bigl[ &f(x_0) + 4f(x_1) + 2f(x_2) + 4f(x_3) + 2f(x_4) + \cdots\\
    &+ 2f(x_{n-2}) + 4f(x_{n-1}) + f(x_n) \bigr].
  \end{split}
  
  

<b> Exercises 1 and 6 from https://tobydriscoll.net/fnc-julia/localapprox/integration.html#exercises): </b>
For $m=5\cdot 2^k, k=1,2,\dots,10$, use $2m+1$ interior points to evaluate the following integrals using both the Trapezoid as well as the Simpson scheme. For each function, plot the error as a function of $k$ for each integration scheme.

 $I_1= \int_0^1 x\log(1+x)\, dx = \frac{1}{4}$
 
 $I_2 =\int_0^1 \sqrt{1-x^2}\,\, dx = \frac{\pi}{4}$
